# IBM DATA SCIENCE PROFESSIONAL CERTIFICATE CAPSTONE PROJECT - PART 1️⃣

# Introduction 🎢

This is the first of two Jupyte Notebooks for the so called *Capstone Project - The Battle of Neighborhoods* developed and teached by IBM on Coursera. It comprises assignment solution for Week 1 and 2.

# Coding Fun 🏖️💻

_Let's import the usual suspects_ 🧙‍♀️🧛🏻‍♂️🧟‍♂️ 🤓:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time # simple timer
%matplotlib inline

_... and some new friends_  🦕🐢🐙🐿️:

In [2]:
import geocoder
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if not installed on your system
from geopy.geocoders import Nominatim # convert an address into latitude and l ongitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas d ataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if not installed on your system
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# 👆🏻 end of week 1️⃣ assignment

---

# Segmenting and Clustering Neighborhoods in Toronto

> ✍️ Assignment:
> 1. Start by creating a new Notebook for this assignment.
> 2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

##### Load list of Postal Codes of Toronto from Wikipedia

In [3]:
# only scrape from wikipedia if you haven't donwlad already download the data

try:
    df = pd.read_pickle("data/df_raw.pkl")
    
except:
    url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
    df = pd.read_html(url)[0] #scrape from wikipedia
    df.to_pickle("data/df_raw.pkl")
    
df.shape

(287, 3)

In [4]:
df.head(3)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


Get rid off 🔫 that stupid `Not assigned` and fill it with `NaN`:

In [5]:
df = df.replace("Not assigned", np.nan)
df.head(2)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN


In [6]:
# How many missing values?
df.isna().sum()

Postcode          0
Borough          77
Neighbourhood    77
dtype: int64

#### a) Remove data without Boroughs

>✍️ Assignment: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
# delete rows with NaN borough
df = df[~df["Borough"].isna()]
print(df.shape)
df.head(2)

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


#### b) Clean Neigbourhood Column
>✍️ Assignment: More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

_This stupid task took me almost 1 day!_ 😤 _to figure out how - trying different approaches!!! Nothing really worked out._ 😠 💣🔪🔫 _Now, I hate Pandas!_ 🐼

In [8]:
# using pandas built-in aggregate function did the trick
df = df.groupby('Postcode').agg(Bourough=("Borough","first"), 
                                 Neighbourhood=('Neighbourhood', list)).reset_index()
df.shape

(103, 3)

In [9]:
df.head(3)

,Postcode,Bourough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"


#### c) Replace not assigned Neigbourhoods with Bouroughs
> ✍️ Assignment: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [10]:
# How many missing values do we have now?
df.isna().sum()

Postcode         0
Bourough         0
Neighbourhood    0
dtype: int64

🤔❓_There are no neighbourshoods with no values. Or other speaking: all Boroughs with `NaN` have also Neighoburhoods with `NaN` and we already removed them in a). So I dont't understand this assignment. Anyhow, in case I miss something and you need to do this: the code to accomplish this would be like this - go crazy_ 🐒:

In [51]:
#replace nan in Neigbourhood with value in Bourough
df['Neighbourhood'].fillna(value=df['Bourough'],inplace=True)
df.head(3)

,Postcode,Bourough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"


In [11]:
df.shape

(103, 3)

### 👆🏻 end of week 2️⃣ - 1️⃣st assignment 
___

In [12]:
# Save DataFrame as file 
df.to_pickle("data/df_cleaned.pkl")

## Adding geo coordinates to Neighbourhoods

_Let's read the saved DataFrame first:_ 📖👓

In [13]:
df = pd.read_pickle("data/df_cleaned.pkl")

✍️ Assignment: 
> Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.
> In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.
> The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. ...
>Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

_Okay. I admit: I'm lazy._ 🛌 _So I use the prepared table_ 👆🏻 _with the coordinates. It's in the `data` folder. I downloaded it, so we can read it in:_

In [14]:
# download table from "https://cocl.us/Geospatial_data" as csv and read it in
df_coord = pd.read_csv("data/Geospatial_Coordinates.csv")
df_coord.head(2)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


_Nice, now we have geo coordinates of our Postal Codes and therefore of our Neighbourhoods. I think, we can do some fancy stuff with this information._ 🕺🏻 But before, we have to merge both datasets .. 👨‍❤️‍💋‍👨

In [16]:
df_coord.shape

(103, 3)

Rename Colmn

In [17]:
# rename column "Postal Code" -> "Postcode" for merging
df_coord.rename(columns={"Postal Code": "Postcode"}, inplace=True)

df_coord.head(1)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353


Merge both Dataframes

In [18]:
df = pd.merge(df, df_coord, how='inner', on='Postcode')

df.head()

,Postcode,Bourough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


In [19]:
df.shape

(103, 5)

--- 

## Explore and cluster the neighborhoods in Toronto

✍️ Assignment:
>Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.   Just make sure:   
>- to add enough Markdown cells to explain what you decided to do and to report any observations you make.   
>- to generate maps to visualize your neighborhoods and how they cluster together.

### Download Foursquare Data for Toronto

[Foursquare](https://foursquare.com) is a social location platform for sharing location based data, e.g.: restaurants, shops, gyms, mueseums, etc. - together with geocodes and additional information like reviews, tips, category, etc.. In order to use Foursquares REST API you need to open a (free) [developer account](https://developer.foursquare.com). After this you receive your own `client_id` and `client_secret` to access the data:

#### But first let's print a map of Toronto 

_Where is Toronto?_ (Of course, in Canada 🇨🇦, but where exatly? Sorry 🙇🏻‍♂️ Canadians for my ignorance)

In [20]:
 # extracting the average coordinates for the toronto map
long = df.Longitude.mean()
lat = df.Latitude.mean()

# define a world map centered around Toronto with a low zoom level
toronto_map = folium.Map(location=[lat, long], zoom_start=2)
folium.features.CircleMarker(
            [lat, long],
            radius=10,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=1).add_to(toronto_map)

# display world map
toronto_map

# alternatively: use the saved image:
#from IPython.display import Image #<-uncomment
#Image('images/world_map.png') #<-uncomment

_Oh - I didn't know Toronto is so near to the US boarder. Maybe this could be interesting for further investigations ..._ 🤔

## Download Venues Data for every Postcode from Foursquare

In [21]:
# Read my Login Information for Foursquare (I stored them in a json file)
# (you can get yours by applying for a free developer account)

file = "data/my_secret_foursquare_login_data.json"
with open (file) as f:
    data = f.read()
login_data = json.loads(data)

# .. and store it in variables
CLIENT_ID = login_data["CLIENT_ID"] # ✍️ paste your client_id here
CLIENT_SECRET = login_data["CLIENT_SECRET"] # ✍️ paste your client_secret instead
VERSION = login_data["VERSION"] # ✍️ paste your version, I used: 20180605

In [22]:
# Function to scrape the venue data thru Foursquares API

def getNearbyVenues(postcodes, latitudes, longitudes, radius=500, LIMIT=200):
    
    venues_list=[]
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# get venues by calling above function

# Scraping took almost 2min - I only run this once and saved the result in data/venues.pkl for later
try: 
    venues=pd.read_pickle("data/venues.pkl")
except:
    start_timer = time.time() #time the scraping
    venues = getNearbyVenues(postcodes=df['Postcode'],
                             latitudes=df['Latitude'],
                             longitudes=df['Longitude']
                            )

    stop_timer = time.time()
    print("Finished! It took {}s to get the venues.".format(round( (stop_timer-start_timer) ,1) ))

    # save venues (so I dont have to scrape again next time)
    venues.to_pickle("data/venues.pkl")
    
venues.shape

(2225, 7)

In [24]:
venues.head(3)

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


_So we now have venues_ 🏢🏫🎡🏟🏦 _of Toronto with their geo coordinates. And we still have a DataFrame with our Toronto Neighbourhoods_  🗺️_. The next step is to merge both:_

In [30]:
# add boroughs to the dataset
#dx = df[["Postcode", "Borough", "Neighbourhood"]]
df_with_venues = pd.merge(venues, df, how="inner", on="Postcode")
df_with_venues.head(3)

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Bourough,Neighbourhood,Latitude,Longitude
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711


In [31]:
df_with_venues.shape

(2225, 11)

In [32]:
# Let's save this
df_with_venues.to_pickle("data/df_with_venues.pkl")

### 👆🏻 end of week 2️⃣ - 2️⃣nd assignment 
___

In [33]:
# Load the data again
df = pd.read_pickle("data/df_with_venues.pkl")

In [34]:
df.shape

(2225, 11)

## Explore Toronto Neighbourhoods and Boroughs

How many categories do we have?

In [35]:
print('There are {} uniques categories.'.format(len(df['Venue Category'].unique())))

There are 267 uniques categories.


_Let's find out what every Borough has to offer ..._ 👐

In [36]:
by_borough = df.groupby("Bourough") # create group object -> every row is now one borough

# How many Postcodes has every Borough?
print("Number of Postcodes per Borough:")
print("===============================")
by_borough.count()["Postcode"].sort_values(ascending=False)

Number of Postcodes per Borough:


Bourough
Downtown Toronto    1307
North York           244
West Toronto         171
East Toronto         127
Central Toronto      112
Scarborough           90
East York             76
Etobicoke             68
York                  17
Mississauga           13
Name: Postcode, dtype: int64

In [37]:
# How many venue types has every Borough?
print("How many venue types does every Borough have?")
print("=============================================")
#by_borough.count()["Venue Category"].sort_values()
by_borough["Venue Category"].nunique().sort_values(ascending=False)

How many venue types does every Borough have?


Bourough
Downtown Toronto    206
North York          108
West Toronto         83
East Toronto         67
Central Toronto      61
Scarborough          55
East York            47
Etobicoke            39
York                 15
Mississauga          10
Name: Venue Category, dtype: int64

---
_Let's do some more advanced stuff..._    
Clustering .. 😱     
In order to cluster the venues (together with their location data) into a few cluster, we need to rebuild the data: 

In [41]:
#every venue gets its own binary variable -> a.k.a one hot encoding
df_onehot = pd.get_dummies(df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Bourough'] = df['Bourough'] 
df_onehot['Neighbourhood'] = df['Neighbourhood'] 
# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])

df_onehot = df_onehot[fixed_columns]

df_onehot.head(3)

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Bourough
0,"[Rouge, Malvern]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Scarborough
1,"[Highland Creek, Rouge Hill, Port Union]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Scarborough
2,"[Guildwood, Morningside, West Hill]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Scarborough


_... some sanity checking:_

In [44]:
# we already did this before
print("Shape: ", df_onehot.shape)
print( df.groupby("Bourough")["Postcode"].count().sort_values(ascending=False).head(2))
print( df.groupby("Bourough")["Venue Category"].nunique().sort_values(ascending=False).head(2))

Shape:  (2225, 269)
Bourough
Downtown Toronto    1307
North York           244
Name: Postcode, dtype: int64
Bourough
Downtown Toronto    206
North York          108
Name: Venue Category, dtype: int64


_Okay, that looks like the ouput before. Let's continue with Clustering_ 🏗... _But first, lets do some more exploratory research:_

In [45]:
df_grouped = df_onehot.groupby('Bourough').mean().reset_index()
np.round(df_grouped.head(4),3)

,Bourough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.018,...,0.000,0.009,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.009
1,Downtown Toronto,0.0,0.001,0.001,0.001,0.001,0.002,0.001,0.001,0.016,...,0.002,0.011,0.002,0.000,0.004,0.000,0.007,0.002,0.001,0.004
2,East Toronto,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.024,...,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.016
3,East York,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.013,0.000,0.013,0.000,0.000,0.000,0.013


#### Let's print each Borough along with the top 5 most common venues

In [47]:
num_top_venues = 5

for borough in df_grouped['Bourough']:
    print("----"+borough+"----")
    temp = df_grouped[df_grouped['Bourough'] == borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1            Park  0.06
2  Sandwich Place  0.06
3            Café  0.05
4             Gym  0.04


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.05
2   Restaurant  0.04
3       Bakery  0.03
4        Hotel  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1  Italian Restaurant  0.05
2         Coffee Shop  0.05
3                Café  0.04
4             Brewery  0.04


----East York----
         venue  freq
0  Coffee Shop  0.05
1         Park  0.05
2  Pizza Place  0.05
3         Bank  0.04
4     Pharmacy  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.10
1  Sandwich Place  0.07
2     Coffee Shop  0.06
3             Gym  0.04
4   Grocery Store  0.04


----Mississauga----
                 venue  freq
0                Hotel  0.15
1         Intersection  0.15
2          Coffee Shop  0.15
3                  Gym  0.08
4  American

_That was pretty cool 🗿, wasn't it? But it's getting better. Fasten your seatbealt ..._ 💺 ⏩

#### Let's put this new information into our **Toronto** dataframe

First, let's write a function to sort the venues in descending order.

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Bourough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Bourough'] = df_grouped['Bourough']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Bourough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Gym,Sushi Restaurant,Dessert Shop,Restaurant,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Bakery,Japanese Restaurant,Hotel,Italian Restaurant,Bar,Park,Seafood Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Café,Pizza Place,Park,Ice Cream Shop,Pub,Sandwich Place
3,East York,Pizza Place,Park,Coffee Shop,Sandwich Place,Pharmacy,Bank,Sporting Goods Shop,Burger Joint,Gym,Skating Rink
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Gym,Fast Food Restaurant,Pharmacy,Grocery Store,Beer Store,Bakery,Convenience Store


## Cluster Analysis

Let's run *k*-means to cluster the bouroughs into clusters.

> 🤓 *k*-means is unsupervised, i.e. it can find clusters by itself (no need for target data and/or splitting the data into train and test set). Anyhow, the drawback of *k*-means is that I have to choose the number of clusters. _(There are some other cluster-algorithm or methods to overcome this, but I leave it for now with setting the number of cluster to 5)._

In [55]:
# set number of clusters
kclusters = 5 # No real argument why I choose 5 

toronto_grouped_clustering = df_grouped.drop('Bourough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 0, 0, 1, 3, 4, 3, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [56]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = manhattan_data
toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Bourough'), on='Bourough')

toronto_merged.head() # check the last columns!

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Bourough,Neighbourhood,Latitude,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,Scarborough,"[Rouge, Malvern]",43.806686,...,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,...,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store
2,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,...,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store
3,M1E,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,...,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store
4,M1E,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,...,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store


In [57]:
# How many Postcodes per Cluster?
y = toronto_merged.copy()
print("Number of Postcodes per Cluster")
print("===============================")
for i in range(kclusters):
    print("Cluster ",i,":" ,y[y["Cluster Labels"]==i].shape[0])

Number of Postcodes per Cluster
Cluster  0 : 144
Cluster  1 : 13
Cluster  2 : 17
Cluster  3 : 1961
Cluster  4 : 90


_Finally, let's visualize_ 🖌️_the resulting clusters:_

In [58]:
toronto_merged.shape

(2225, 22)

In [59]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Bourough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Screenshot of above map:
![Image of Toronto with Clusters](images/toronto_map_with_clusters.png)

From the map 👆🏻 we can see two things:
1. Cluster were built mainly based on coordinates, but not only (also on venue categories). 
2. There is one cluster which is only consisting of one case and therefore pretty useless 🔫.

### Building subsets for each cluster for further investigation

In [60]:
# Lets create sub-dataframes for each cluster for better investigation
clusters = {}
for i in range(kclusters+1):
    clusters[i] = toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(i, toronto_merged.shape[1]))]]

In [66]:
clusters[3]

,Neighborhood Latitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Bourough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,43.803762,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course,North York,[Hillcrest Village],43.803762,-79.363452,3,Coffee Shop,Clothing Store,Japanese Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Café,Park,Grocery Store,Pizza Place
91,43.803762,New York Fries,43.803664,-79.363905,Fast Food Restaurant,North York,[Hillcrest Village],43.803762,-79.363452,3,Coffee Shop,Clothing Store,Japanese Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Café,Park,Grocery Store,Pizza Place
92,43.803762,AY Jackson Pool,43.804515,-79.366138,Pool,North York,[Hillcrest Village],43.803762,-79.363452,3,Coffee Shop,Clothing Store,Japanese Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Café,Park,Grocery Store,Pizza Place
93,43.803762,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant,North York,[Hillcrest Village],43.803762,-79.363452,3,Coffee Shop,Clothing Store,Japanese Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Café,Park,Grocery Store,Pizza Place
94,43.803762,Duncan Creek Park,43.805539,-79.360695,Dog Run,North York,[Hillcrest Village],43.803762,-79.363452,3,Coffee Shop,Clothing Store,Japanese Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Café,Park,Grocery Store,Pizza Place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2151,43.662744,ONE Academy,43.662253,-79.326911,Gym / Fitness Center,East Toronto,[Business Reply Mail Processing Centre 969 Eas...,43.662744,-79.321558,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Café,Pizza Place,Park,Ice Cream Shop,Pub,Sandwich Place
2198,43.756303,Pizza Monza,43.755043,-79.567195,Pizza Place,North York,[Humber Summit],43.756303,-79.565963,3,Coffee Shop,Clothing Store,Japanese Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Café,Park,Grocery Store,Pizza Place
2199,43.756303,The Famous Mama Mia,43.758820,-79.570637,Empanada Restaurant,North York,[Humber Summit],43.756303,-79.565963,3,Coffee Shop,Clothing Store,Japanese Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Café,Park,Grocery Store,Pizza Place
2200,43.724766,Strathburn Park,43.721765,-79.532854,Baseball Field,North York,"[Emery, Humberlea]",43.724766,-79.532242,3,Coffee Shop,Clothing Store,Japanese Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Café,Park,Grocery Store,Pizza Place


In [ ]:
# Finally, let's save our new information


_Now it would be a fantastic time to look more deeper into these clusters and try to interpret them (good luck 🍀 with that)_.   
_I don't want to go into to much detail (as I understand the second part of the assignment is for analyzing and finding results) and leave it here ..._

# 👆🏻 end of week 2️⃣ - 3️⃣rd and last assignment

---
# 🔚 🙏🏼 🙋🏻‍♂️

In [62]:
## _See U in Part_ 2️⃣